In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

In [11]:
df = pd.read_csv('./data/validation/한미반도체.csv')
df.rename(columns={'GSTC_CODE': 'code',
                   'STCK_PRPR': '종가',
                   'CNTG_VOL': '거래량'},
          inplace=True)

In [12]:
stock_number = 96993634
df['거래량'] = df['거래량'] / stock_number * 1000000

In [13]:
df.to_csv('./data/validation/한미반도체allTemp.csv', index=False)

In [5]:

"""
# ##    정렬
# df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d')
# df = df.sort_values(by=['code','날짜','시간'])
# df = df.reset_index(drop=True)
# df = df.iloc[:,1:]

# ##    난수 처리
# df = df.dropna(subset=['MA120'])

# df.to_csv('all.csv')

print(df.head())

##      거래량 스케일링
financeList = pd.read_csv('FinanceListPreprocess.csv')
name = financeList['종목코드'].tolist()
volSum = financeList['상장주식수'].tolist()
finance_dict = dict(zip(name, volSum))
df = df.iloc[:, 1:]
df['code'] = df['code'].astype(str).str.zfill(6)
"""


def divide_volumes(row):
    try:
        divisor = 15054186
        row[['거래량']] = row[['거래량']] * 1000000 / divisor
        row['거래량'] = round(row['거래량'], 2)
    except KeyError:
        print(f"코드 {row['code']}를 찾지 못했습니다.")
        input()
    return row

output_file = 'allTemp.csv'
df.head(0).to_csv(output_file, index=False)
cnt = 0

for idx, row in df.iterrows():
    processed_row = divide_volumes(row)
    processed_row.to_frame().T.to_csv(output_file, index=False, mode='a', header=False)
    if cnt % 1000 == 0:
        print(f'{cnt}개 처리 완료')
    cnt += 1

print("데이터 처리 완료")



0개 처리 완료
1000개 처리 완료
2000개 처리 완료
3000개 처리 완료
4000개 처리 완료
5000개 처리 완료
6000개 처리 완료
7000개 처리 완료
8000개 처리 완료
9000개 처리 완료
10000개 처리 완료
11000개 처리 완료
12000개 처리 완료
13000개 처리 완료
14000개 처리 완료
15000개 처리 완료
16000개 처리 완료
17000개 처리 완료
18000개 처리 완료
19000개 처리 완료
20000개 처리 완료
21000개 처리 완료
22000개 처리 완료
23000개 처리 완료
24000개 처리 완료
25000개 처리 완료
26000개 처리 완료
27000개 처리 완료
28000개 처리 완료
29000개 처리 완료
30000개 처리 완료


KeyboardInterrupt: 

In [14]:
# 데이터 로드
# stock_names = ['CJ제일제당', 'KT', 'SK텔레콤', 'SK하이닉스', '농심', '삼성전자', '삼양식품', '한미반도체']
stock_names = ['한미반도체']
for stock_name in stock_names:
    df = pd.read_csv(f'./data/validation/{stock_name}allTemp.csv')
    df=df[['code','종가', '거래량']]
    df.rename(columns={'종가' : 'close','거래량' : 'volume',}, inplace=True)


    # Unique 코드 및 카운트 계산
    counts = df['code'].value_counts()
    counts_dict = counts.to_dict()

    # 결과를 저장할 리스트
    results = []

    for key in tqdm(counts_dict.keys()):
        value = counts_dict[key]
        for i in range(value - 12):   # 예측 분 -1
            start_idx = counts_dict[key] - value + i
            end_idx = start_idx + 9  # 묶을 분 -1

            # 벡터화하여 연산
            vectorList = []
            for col in ['close','volume']:
                diff = df[col].values[start_idx:end_idx] - df[col].values[start_idx + 1:end_idx + 1]
                vectorList.extend(round(d, 2) for d in diff)

            openValue = df['close'][start_idx + 12] - df['close'][start_idx + 9]
            if openValue > 0:
                vectorList.append(2.0)
            elif openValue == 0:
                vectorList.append(1.0)
            else:
                vectorList.append(0.0)

            results.append(vectorList)

    # 결과를 DataFrame으로 변환
    results_df = pd.DataFrame(results)

    # CSV 파일에 저장
    results_df.to_csv(f'./data/validation/preprocessed_{stock_name}.csv', index=False)
'''
##      난수 및 의미 없는 데이터 삭제
df = pd.read_csv('allVector.csv')
df.columns = [f"Column{i+1}" for i in range(df.shape[1])]
mask = (df.iloc[:, :-1] == 0.0).all(axis=1)
num_zero_rows = mask.sum()
print(f"마지막 열의 값을 제외하고 다른 값들이 모두 0.0인 행의 갯수: {num_zero_rows}")
df = df[~mask]
df = df.dropna()
 
##    라벨 비율 맞추기
value_counts = df['Column53'].value_counts()
print("각 값의 갯수:")
print(value_counts)

min_count = value_counts.min()
balanced_df_list = []
for value in value_counts.index:
    subset = df[df['Column53'] == value].sample(n=min_count, random_state=42)
    balanced_df_list.append(subset)
df = pd.concat(balanced_df_list).reset_index(drop=True)
print(len(df))
df.to_csv('readyAll1.csv')
'''

100%|██████████| 1/1 [00:00<00:00, 112.48it/s]


'\n##      난수 및 의미 없는 데이터 삭제\ndf = pd.read_csv(\'allVector.csv\')\ndf.columns = [f"Column{i+1}" for i in range(df.shape[1])]\nmask = (df.iloc[:, :-1] == 0.0).all(axis=1)\nnum_zero_rows = mask.sum()\nprint(f"마지막 열의 값을 제외하고 다른 값들이 모두 0.0인 행의 갯수: {num_zero_rows}")\ndf = df[~mask]\ndf = df.dropna()\n \n##    라벨 비율 맞추기\nvalue_counts = df[\'Column53\'].value_counts()\nprint("각 값의 갯수:")\nprint(value_counts)\n\nmin_count = value_counts.min()\nbalanced_df_list = []\nfor value in value_counts.index:\n    subset = df[df[\'Column53\'] == value].sample(n=min_count, random_state=42)\n    balanced_df_list.append(subset)\ndf = pd.concat(balanced_df_list).reset_index(drop=True)\nprint(len(df))\ndf.to_csv(\'readyAll1.csv\')\n'

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import BatchNorm1d
from sklearn.metrics import confusion_matrix, classification_report
from torch.optim.lr_scheduler import StepLR

print(f'GPU available : {torch.cuda.is_available()}')

GPU available : True


In [18]:
# 데이터 로드
df = pd.read_csv('C:/big18/dl-dev/dl-dev/project/allVector.csv')
# df = df.iloc[:,2:]
# df = df.astype('float16')

print(df)
# 노드 특징 행렬 (Node feature matrix)
x = torch.tensor(df.iloc[:,:-1].values, dtype=torch.float)

# 엣지 리스트 (Edge list)
edge_index = []
num_rows = len(df)
for i in range(num_rows - 1):
    edge_index.append([i, i + 1])
    edge_index.append([i + 1, i])

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# 레이블
y = torch.tensor(df['8'].values, dtype=torch.long)

# 그래프 데이터 객체
# graph_data = Data(x=x, edge_index=edge_index, y=y)

# 데이터셋 분리
train_indices, val_indices = train_test_split(range(num_rows), test_size=0.2, random_state=42)
train_mask = torch.zeros(num_rows, dtype=torch.bool)
val_mask = torch.zeros(num_rows, dtype=torch.bool)
train_mask[train_indices] = 1
val_mask[val_indices] = 1

train_data = Data(x=x, edge_index=edge_index, y=y)
train_data.train_mask = train_mask
train_data.val_mask = val_mask

          0    1    2    3      4      5      6      7    8
0      -200  800 -100 -100  59.83  24.80  12.91  12.31  0.0
1       800 -100 -100  100  24.80  12.91  12.31  16.17  0.0
2      -100 -100  100  200  12.91  12.31  16.17 -20.34  1.0
3      -100  100  200    0  12.31  16.17 -20.34   0.81  2.0
4       100  200    0  100  16.17 -20.34   0.81  14.80  2.0
...     ...  ...  ...  ...    ...    ...    ...    ...  ...
186827    0 -300  100    0  14.29  -3.82   0.61  -2.47  2.0
186828 -300  100    0 -100  -3.82   0.61  -2.47   2.43  0.0
186829  100    0 -100  100   0.61  -2.47   2.43   0.28  0.0
186830    0 -100  100    0  -2.47   2.43   0.28 -13.39  0.0
186831 -100  100    0 -100   2.43   0.28 -13.39   9.63  1.0

[186832 rows x 9 columns]


In [19]:
#      복잡 모델
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.bn1 = BatchNorm1d(16)
        self.conv2 = GCNConv(16, 32)
        self.bn2 = BatchNorm1d(32)
        self.conv3 = GCNConv(32, 64)
        self.bn3 = BatchNorm1d(64)
        self.conv4 = GCNConv(64, num_classes)
        # self.bn4 = BatchNorm1d(128)
        # self.conv5 = GCNConv(128, num_classes)
        # self.bn5 = BatchNorm1d(256)
        # self.conv6 = GCNConv(256, num_classes)
        # self.bn6 = BatchNorm1d(512)
        # self.conv7 = GCNConv(512, num_classes)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.gelu(x)
        # x = self.dropout(x)

        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.gelu(x)
        # x = self.dropout(x)

        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.gelu(x)
        x = self.dropout(x)
        
        x = self.conv4(x, edge_index)
        # x = self.bn4(x)
        # x = F.gelu(x)
        # x = self.dropout(x)
        
        # x = self.conv5(x, edge_index)
        # x = self.bn5(x)
        # x = F.gelu(x)
        # x = self.dropout(x)
        
        # x = self.conv6(x, edge_index)
        # x = self.bn6(x)
        # x = F.gelu(x)
        # x = self.dropout(x)
        
        # x = self.conv7(x, edge_index)

        return F.log_softmax(x, dim=1)

# 모델 초기화
model = GCN(num_node_features=x.size(1), num_classes=len(y.unique()))

In [20]:
import torch.optim as optim

# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_data = train_data.to(device)

# 옵티마이저 정의
optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)  #   optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)  # 학습률&정규화 조정 (model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.9)

# 조기 종료 설정
early_stopping_patience = 100    # *10
early_stopping_counter = 0
best_acc = 0.0

# 정확도 계산 함수
def accuracy(pred, labels):
    _, pred_classes = pred.max(dim=1)
    correct = (pred_classes == labels).sum().item()
    return correct / len(labels)

# 모델 훈련 및 평가
model.train()
for epoch in range(10000):
    optimizer.zero_grad()
    out = model(train_data)
    loss = F.nll_loss(out[train_data.train_mask], train_data.y[train_data.train_mask])
    loss.backward()
    optimizer.step()
    scheduler.step()

    # 평가 모드로 전환하여 테스트 셋에서의 성능을 평가
    model.eval()
    with torch.no_grad():
        val_out = model(train_data)
        val_loss = F.nll_loss(val_out[train_data.val_mask], train_data.y[train_data.val_mask])
        val_acc = accuracy(val_out[train_data.val_mask], train_data.y[train_data.val_mask])

    # 모델을 다시 훈련 모드로 전환
    model.train()

    if epoch % 10 == 0:
        train_acc = accuracy(out[train_data.train_mask], train_data.y[train_data.train_mask])
        current_lr = scheduler.get_last_lr()[0]
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.4f}     Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_acc:.4f}    Lr : {current_lr:.6f}')
        if epoch > 500:
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), 'gcn.pth')
                early_stopping_counter = 0
                print(f'New best model saved with val accuracy: {best_acc:.4f}')
            else:
                early_stopping_counter += 1
    if early_stopping_counter >= early_stopping_patience:
        print(f'Early stopping at epoch {epoch}')
        break

Epoch 0, Loss: 1.1562, Train Accuracy: 0.3733     Val Loss: 5.8250, Val Accuracy: 0.4244    Lr : 0.100000
Epoch 10, Loss: 0.9698, Train Accuracy: 0.5346     Val Loss: 0.9904, Val Accuracy: 0.5166    Lr : 0.100000
Epoch 20, Loss: 0.8993, Train Accuracy: 0.5726     Val Loss: 1.0015, Val Accuracy: 0.5382    Lr : 0.100000
Epoch 30, Loss: 0.8643, Train Accuracy: 0.5887     Val Loss: 0.9111, Val Accuracy: 0.5509    Lr : 0.100000
Epoch 40, Loss: 0.8392, Train Accuracy: 0.6013     Val Loss: 0.8869, Val Accuracy: 0.5776    Lr : 0.100000
Epoch 50, Loss: 0.8260, Train Accuracy: 0.6129     Val Loss: 0.8979, Val Accuracy: 0.5647    Lr : 0.100000
Epoch 60, Loss: 0.8060, Train Accuracy: 0.6227     Val Loss: 0.9544, Val Accuracy: 0.5152    Lr : 0.100000
Epoch 70, Loss: 0.7825, Train Accuracy: 0.6355     Val Loss: 0.9136, Val Accuracy: 0.5421    Lr : 0.100000
Epoch 80, Loss: 0.7904, Train Accuracy: 0.6377     Val Loss: 0.9202, Val Accuracy: 0.5426    Lr : 0.100000
Epoch 90, Loss: 0.7489, Train Accuracy

In [22]:
# 학습 완료 후 최종 평가
model.load_state_dict(torch.load('C:/big18/dl-dev/dl-dev/gcn.pth'))
model.eval()
with torch.no_grad():
    final_out = model(train_data)
    final_loss = F.nll_loss(final_out[train_data.val_mask], train_data.y[train_data.val_mask])
    final_acc = accuracy(final_out[train_data.val_mask], train_data.y[train_data.val_mask])
    val_out = model(train_data)
    val_pred = val_out[train_data.val_mask].max(dim=1)[1].cpu().numpy()
    val_true = train_data.y[train_data.val_mask].cpu().numpy()

conf_matrix = confusion_matrix(val_true, val_pred)
class_report = classification_report(val_true, val_pred)

print("Confusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)

print(f'Final Eval Loss: {final_loss.item():.4f}, Final Eval Accuracy: {final_acc:.4f}')

C:\Users\F06\AppData\Local\Temp\ipykernel_22572\2458309604.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('C:/big18/dl-dev/dl-dev/gcn.p

Confusion Matrix:
[[8964 3881  965]
 [2302 5421 2577]
 [ 873 3549 8835]]

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.65      0.69     13810
           1       0.42      0.53      0.47     10300
           2       0.71      0.67      0.69     13257

    accuracy                           0.62     37367
   macro avg       0.62      0.61      0.62     37367
weighted avg       0.64      0.62      0.63     37367

Final Eval Loss: 0.8692, Final Eval Accuracy: 0.6214
